### Домашняя работа

**Задача высокого уровня** В реализацию функции `gradient` добавьте параметр $\lambda$, чтобы получить регуляризованный градиентный спуск

Формула поменяется следующим образом:
$$
\left\{
\begin{array}{cc}
\frac{\partial L}{\partial w_0} = \frac{2}{n}\cdot(-1)\cdot \sum_{i=1}^{n} (1\cdot \left(y_i - \sum_{j=1}^{m}w_jx_j^i\right) + \lambda\cdot 2\cdot w_0)&\\
\frac{\partial L}{\partial w_k} = \frac{2}{n}\cdot(-1)\cdot \sum_{i=1}^{n} (x_k^i \cdot\left(y_i - \sum_{j=1}^{m}w_jx_j^i\right) + \lambda\cdot 2\cdot w_k)& k\neq 0 \\
\end{array}
\right.
$$

In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv('non_linear.csv', sep=',')
data = data[(data.x_train > 1) & (data.x_train < 5)].copy()
data.head()

,x_train,y_train
5,1.182421,1.860341
6,1.251605,1.878928
7,1.270474,2.430015
8,1.402553,2.327856
9,1.427711,2.203649


In [3]:
from scipy.spatial import distance

def gradient(X, y, w, r_coef=0):
    # количество обучающих примеров в выборке
    n = X.shape[0]
    # считаем прогноз
    y_hat = X.dot(w.T)
    # вычисляем ошибку прогноза
    error = y - y_hat
    # дальше pointwise перемножение - умножаем каждую из координат на ошибку
    grad = -2.0/n * (np.multiply(X, error) + r_coef * 2 * w).sum(axis=0)
    return grad, error

def eval_w_next(X, y, eta, w_current, r_coef):
    # вычисляем градиент
    grad, error = gradient(X, y, w_current, r_coef)
    # делаем шаг градиентного спуска
    w_next = w_current - eta*grad
    # проверяем условие сходимости
    weight_evolution = distance.euclidean(w_current, w_next)
    return (w_next, weight_evolution, grad)

def gradient_descent(X: np.array, y: np.array, eta=0.01, epsilon=0.001, r_coef=0) -> np.array:
    m = X.shape[1] # количество фичей
    # инициализируем рандомом веса
    w = np.random.random(m).reshape(1, -1)
    w_next, weight_evolution, grad = eval_w_next(X, y, eta, w, r_coef)
    step = 0
    # повторяем до сходимости вектора весов
    while weight_evolution > epsilon:
        w = w_next
        w_next, weight_evolution, grad = eval_w_next(X, y, eta, w, r_coef)
        step += 1
        if step % 100 ==0:
            print("step %s |w-w_next|=%.5f, grad=%s" % (step, weight_evolution, grad))
    return w

In [4]:
# трансформируем плоский массив X в вектор-столбец
X = data['x_train'].values.reshape(-1, 1)
n = X.shape[0]
# добавляем тривиальный признак w_0, столбец из единиц. См. прошлый урок, почему так
X = np.hstack([
    np.ones(n).reshape(-1,1),
    X
])
w = gradient_descent(X, data['y_train'].values.reshape(-1, 1), eta=0.008, r_coef=0.02)

step 100 |w-w_next|=0.00538, grad=[-0.6459346   0.18659769]
step 200 |w-w_next|=0.00472, grad=[-0.56674719  0.16372199]
step 300 |w-w_next|=0.00414, grad=[-0.49726765  0.14365073]
step 400 |w-w_next|=0.00363, grad=[-0.43630584  0.12604008]
step 500 |w-w_next|=0.00319, grad=[-0.38281756  0.11058838]
step 600 |w-w_next|=0.00280, grad=[-0.3358866   0.09703096]
step 700 |w-w_next|=0.00245, grad=[-0.29470907  0.08513559]
step 800 |w-w_next|=0.00215, grad=[-0.25857964  0.07469851]
step 900 |w-w_next|=0.00189, grad=[-0.22687944  0.06554096]
step 1000 |w-w_next|=0.00166, grad=[-0.19906549  0.05750606]
step 1100 |w-w_next|=0.00145, grad=[-0.17466134  0.05045619]
step 1200 |w-w_next|=0.00128, grad=[-0.15324899  0.04427059]
step 1300 |w-w_next|=0.00112, grad=[-0.13446165  0.0388433 ]


В этом модуле мы узнали, как  обучать линейную регрессию, не "упираясь" в аппаратные ресурсы: использовать градиентный спуск.
Мы узнали, как детектировать переобучение модели и закрепили свои знания на примере полиномиальной регрессии и выяснили, как увеличить качество решения с помощью механизма регуляризации. Познакомились с двумя видами регуляризации -  Ridge и Lasso.